# Human monitoring activity

In [1]:
import seaborn as sns

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression


import tensorflow as tf
from tensorflow  import keras as k
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

# # Read data & add to X, y

In [2]:
def load_files(filepath):
    dataset = pd.DataFrame()
    #y = 
    for i in range(1, 15):
        filename1 = filepath + str(i) + '.csv'
        # I need class as a column to be able to do a pairplot
        dataset_trainX = pd.read_csv(filename1, usecols=[1,2,3,4], names=['ax', 'ay', 'az', 'class'], header=None)
        dataset = dataset.append(dataset_trainX, ignore_index = True)
        #position_class

    return dataset

In [3]:
filepath = 'activityRecognition/'
dataset = load_files(filepath)

In [4]:
#dataset["class"].plot()
dataset.describe()

,ax,ay,az,class
count,1.823395e+06,1.823395e+06,1.823395e+06,1.823395e+06
mean,1.982025e+03,2.374125e+03,1.968226e+03,3.938922e+00
std,1.112281e+02,9.555790e+01,9.515460e+01,2.441348e+00
min,2.820000e+02,2.000000e+00,1.000000e+00,0.000000e+00
25%,1.900000e+03,2.334000e+03,1.915000e+03,1.000000e+00
50%,1.981000e+03,2.362000e+03,1.985000e+03,4.000000e+00
75%,2.070000e+03,2.395000e+03,2.031000e+03,7.000000e+00
max,3.828000e+03,4.095000e+03,4.095000e+03,7.000000e+00


In [5]:
dataset = dataset[dataset['class'] != 0] # tar bort alla med klass 0

In [6]:
# test data model function 
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [7]:
#dataset["class"].plot() # kolla så allt stämmer dvs klasserna ligger mellan 1-7

In [8]:
#dataset.drop('class', inplace=False, axis=1).plot()

# Remove outliners

In [9]:
# reomve ouliners # ta bort alla extrema värden!

#ay
max_thresold = dataset["ay"].quantile(0.99997) #0.999985  lägre mindre distru

min_thresold = dataset["ay"].quantile(0.000015)  # högre mindre dis -1->5---jag har nu lag högre

#az
az_max_thresold = dataset["az"].quantile(0.999982) #0.999985  lägre mindre dis

az_min_thresold = dataset["az"].quantile(0.00004)  # högre mindre distru -1->5---jag har nu lag högre

ax_min_thresold = dataset["ax"].quantile(0.00002)  # högre mindre distru -1->5---jag har nu lag högre
#X = X[(X["ay"]<max_thresold) & (X["ay"]>min_thresold)]
#X.plot()

dataset_noEx = pd.DataFrame()

dataset_noEx = dataset[(dataset["ay"]<max_thresold) & (dataset["ay"]>min_thresold) & (dataset["az"]<az_max_thresold) & (dataset["az"]>az_min_thresold)  & (dataset["ax"]>ax_min_thresold)]

dataset_noEx.head()
dataset_noEx.describe()

,ax,ay,az,class
count,1.819520e+06,1.819520e+06,1.819520e+06,1.819520e+06
mean,1.981830e+03,2.373882e+03,1.968129e+03,3.947217e+00
std,1.108444e+02,9.465487e+01,9.438587e+01,2.437288e+00
min,1.115000e+03,1.042000e+03,1.154000e+03,1.000000e+00
25%,1.900000e+03,2.334000e+03,1.915000e+03,1.000000e+00
50%,1.980000e+03,2.362000e+03,1.985000e+03,4.000000e+00
75%,2.070000e+03,2.394000e+03,2.031000e+03,7.000000e+00
max,3.154000e+03,3.120000e+03,3.124000e+03,7.000000e+00


In [10]:
#sns_plot = sns.pairplot(dataset_noEx, hue="class", palette="coolwarm")

# Split data

In [11]:
#split data to x y.
X = dataset_noEx.drop('class', inplace=False, axis=1)
y = dataset_noEx.drop(['ax', 'ay', 'az'], inplace=False, axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Standard scalar

In [12]:
from sklearn.preprocessing import StandardScaler
# standard scale  ???? ska man???
sc=StandardScaler()
train_x=sc.fit_transform(X_train)
test_x=sc.fit_transform(X_test)

# KNN

In [13]:
Y = y_train["class"]
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=65) 
classifier.fit(train_x, y_train)

C:\Users\amina\anaconda3\envs\env-tf\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=65)

In [14]:
y_pred = classifier.predict(test_x)
from sklearn.metrics import classification_report, confusion_matrix

#print(classification_report(y_test, y_pred))
"""
print(confusion_matrix(y_test, y_pred))
MSE = np.square(np.subtract(y_test["class"], y_pred)).mean()
import math
RMSE = math.sqrt(MSE)
print(RMSE)

from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
print(RMSE)
#accuracy_score(actual, predicted)
"""

'\nprint(confusion_matrix(y_test, y_pred))\nMSE = np.square(np.subtract(y_test["class"], y_pred)).mean()\nimport math\nRMSE = math.sqrt(MSE)\nprint(RMSE)\n\nfrom sklearn.metrics import mean_squared_error\nMSE = mean_squared_error(y_test, y_pred)\nRMSE = math.sqrt(MSE)\nprint(RMSE)\n#accuracy_score(actual, predicted)\n'

In [15]:
"""X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.preprocessing import StandardScaler
best = 0 #testing the best nr for k
lastAcc = 0
listAcc = []
for x in range(1,10):
    # standard scale  ???? ska man???
    sc=StandardScaler()
    train_x=sc.fit_transform(X_train)
    test_x=sc.fit_transform(X_test)

    Y = y_train["class"]
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=x) # testa dig fram till bästa resulat- skapa en for loop som kollar
    classifier.fit(X_train, Y)

    acc = get_score(classifier, X_train, X_test, y_train, y_test)
    if acc > lastAcc:
        best = x
        lastAcc = acc
    keeper ="K: ", x, "Acc: ", acc
    listAcc.append(keeper)
    
    #accuracy_score(actual, predicted)
listAcc
best
"""

'X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)\nfrom sklearn.preprocessing import StandardScaler\nbest = 0 #testing the best nr for k\nlastAcc = 0\nlistAcc = []\nfor x in range(1,10):\n    # standard scale  ???? ska man???\n    sc=StandardScaler()\n    train_x=sc.fit_transform(X_train)\n    test_x=sc.fit_transform(X_test)\n\n    Y = y_train["class"]\n    from sklearn.neighbors import KNeighborsClassifier\n    classifier = KNeighborsClassifier(n_neighbors=x) # testa dig fram till bästa resulat- skapa en for loop som kollar\n    classifier.fit(X_train, Y)\n\n    acc = get_score(classifier, X_train, X_test, y_train, y_test)\n    if acc > lastAcc:\n        best = x\n        lastAcc = acc\n    keeper ="K: ", x, "Acc: ", acc\n    listAcc.append(keeper)\n    \n    #accuracy_score(actual, predicted)\nlistAcc\nbest\n'

# Confusion matrix

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

confusion_matrix(y_test, y_pred)

array([[103736,    502,    934,   2812,     57,     20,   3545],
       [  3768,   1191,    465,   1913,     25,      9,   1362],
       [  2029,     99,  20050,   8941,    665,    410,   9420],
       [  4625,     72,   3116,  50963,    162,    198,   8458],
       [   801,      4,   1848,   4016,   1207,    130,   1616],
       [   389,      4,   1063,   1737,    376,   1971,   3995],
       [  3562,    101,   4182,   7068,    152,    832,  99303]],
      dtype=int64)

In [ ]:
matrix = plot_confusion_matrix(classifier, test_x, y_test, cmap=plt.cm.Reds)
matrix.ax_.set_title("Confusion Matrix")
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.gcf().set_size_inches(10, 6)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
labels = ["1", "2", "3", "4", "5", "6", "7"]
cm = confusion_matrix(y_test, y_pred)
#print(cm, "CM")
#print(pd.Dataframe(cm))
# Accuracy
from sklearn.metrics import accuracy_score
#print(accuracy_score(y_test, y_pred) , "AC")
# Recall
from sklearn.metrics import recall_score
#print(recall_score(y_test, y_pred, average=None), "RE")
# Precision
from sklearn.metrics import precision_score
#print(precision_score(y_test, y_pred, average=None) , "PR")

# info 
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))